In [2]:
import pandas as pd
import networkx as nx

# Load airport data
airports_df = pd.read_csv("./CSVFiles/airports.csv")  # Contains Airport ID, IATA, Name, Country
routes_df = pd.read_csv("./CSVFiles/routes.csv")  # Contains Source airport, Destination airport

# Filter for US and Canada airports
US_Canada_Airports = airports_df[airports_df["Country"].isin(["Canada", "United States"])]

# Map IATA codes to airport names
airport_name_map = dict(zip(US_Canada_Airports["IATA"], US_Canada_Airports["Name"]))

# Keep only flights where both airports are in US/Canada
filtered_flights = routes_df[
    (routes_df["Source airport"].isin(airport_name_map)) & 
    (routes_df["Destination airport"].isin(airport_name_map))
]

# Normalize flight direction (ensuring undirected edges)
filtered_flights["Route"] = filtered_flights.apply(lambda row: tuple(sorted([row["Source airport"], row["Destination airport"]])), axis=1)

# Count occurrences of each unique route (weight)
route_counts = filtered_flights["Route"].value_counts().reset_index()
route_counts.columns = ["Route", "Weight"]

# Split back into Source and Target
route_counts[["Source", "Target"]] = pd.DataFrame(route_counts["Route"].tolist(), index=route_counts.index)

# Create a Graph
G = nx.Graph()

# Add edges with weight
for _, row in route_counts.iterrows():
    G.add_edge(row["Source"], row["Target"], weight=row["Weight"])
    G.add_node(row["Source"], label = row["Source"])
# Save as GEXF (supported by Gephi)
nx.write_gexf(G, "FinalProject.gexf")

print("Filtered graph file created: FinalProject.gexf")


Filtered graph file created: FinalProject.gexf


C:\Users\ASUS\AppData\Local\Temp\ipykernel_16876\297288731.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_flights["Route"] = filtered_flights.apply(lambda row: tuple(sorted([row["Source airport"], row["Destination airport"]])), axis=1)
